# Chapter 15. Introduction to Multimedia Data
## Notebook for R

### 15.1 Storing, representing, and converting multimedia data

In [1]:
library(magick)
my_image1 = image_read('259_3_32_15.jpg')
my_image2 = image_read('298_5_52_15.png')
print(image_info(my_image1))
print(image_info(my_image2))

Linking to ImageMagick 6.9.9.39
Enabled features: cairo, fontconfig, freetype, lcms, pango, rsvg, webp
Disabled features: fftw, ghostscript, x11



  format width height colorspace matte filesize density
1   JPEG   805    453       sRGB FALSE    75275   72x72
  format width height colorspace matte filesize density
1    PNG  1540    978       sRGB  TRUE  2752059   57x57


In [12]:
my_image1_L = image_convert(my_image1, colorspace = 'gray')
print(class(my_image1_L))
my_image1_L_array = as.integer(my_image1_L[[1]])
print(class(my_image1_L_array))
print(dim(my_image1_L_array))


[1] "magick-image"
[1] "array"
[1] 453 805   1


In [13]:
my_image1_RGB = image_convert(my_image1, colorspace = 'RGB')
print(class(my_image1_RGB))
my_image1_RGB_array = as.integer(my_image1_RGB[[1]])
print(class(my_image1_RGB_array))
print(dim(my_image1_RGB_array))



[1] "magick-image"
[1] "array"
[1] 453 805   3
